# Лабораторная работа №3
## Сжатие фото и видеоизображения
### Логинов Сергей
### Наливайко Сергей
### Гафиров Абдималик
Задание 1:
Реализовать сжатие видео как отдельное сжатие каждого кадра сжатие реализовать одним из двух алгоритмов сжатия изображения
Первый студент в таблице - Наливайко Сергей (6 - последняя цифра студ. номера), соответственно наш вариант - сжатие с потерями, как в формате jpg

In [1]:
import pywt
import numpy as np
import os
import cv2
import glob
from imageio import imread
from PIL import Image
from moviepy.editor import *

В интернете был найден алгоритм сжатия данных, основанный на вейвлет-преобразовании
Некоторые источники:

https://habr.com/ru/articles/168517/

https://ru.wikipedia.org/wiki/Сжатие_с_использованием_вейвлет#:~:text=Вейвлетное%20сжатие%20—%20общее%20название%20класса,подразумевается%20сжатие%20с%20потерей%20качества.

https://core.ac.uk/download/pdf/53065985.pdf


In [2]:
def compress_image(image_path, quality):
    """
    Сжатие изображения с использованием
    вейвлет-преобразования
    """
    image = Image.open(image_path)
    image_array = np.array(image)
    
    # Разделение на каналы цвета
    red_channel = image_array[:, :, 0]
    green_channel = image_array[:, :, 1]
    blue_channel = image_array[:, :, 2]
    
    # Применение вейвлет-преобразования к каждому каналу цвета
    coeffs_red = pywt.dwt2(red_channel, 'haar')
    coeffs_green = pywt.dwt2(green_channel, 'haar')
    coeffs_blue = pywt.dwt2(blue_channel, 'haar')
    
    # Квантование коэффициентов - уменьшение количества информации сохраненной в кэффах
    coeffs_red_quantized = np.round(coeffs_red[0] / (quality / 100.0))
    coeffs_green_quantized = np.round(coeffs_green[0] / (quality / 100.0))
    coeffs_blue_quantized = np.round(coeffs_blue[0] / (quality / 100.0))
    
    # Создание сжатых коэффициентов
    compressed_coeffs = [(coeffs_red_quantized, coeffs_green_quantized, coeffs_blue_quantized),
                         coeffs_red[1], coeffs_green[1], coeffs_blue[1]]
    
    # Определение размера изображения
    image_size = image_array.shape[:2]
    
    return compressed_coeffs, image_size

def decompress_image(compressed_coeffs, image_size, quality):
    """
    Восстановление изображения из сжатых массивов
    """
    # Извлечение сжатых коэффициентов
    coeffs_red_quantized, coeffs_green_quantized, coeffs_blue_quantized = compressed_coeffs[0]
    coeffs_red_details, coeffs_green_details, coeffs_blue_details = compressed_coeffs[1:]
    
    # Восстановление коэффициентов
    coeffs_red = coeffs_red_quantized * (quality / 100.0)
    coeffs_green = coeffs_green_quantized * (quality / 100.0)
    coeffs_blue = coeffs_blue_quantized * (quality / 100.0)
    
    # Восстановление изображения
    red_channel = pywt.idwt2((coeffs_red, coeffs_red_details), 'haar')
    green_channel = pywt.idwt2((coeffs_green, coeffs_green_details), 'haar')
    blue_channel = pywt.idwt2((coeffs_blue, coeffs_blue_details), 'haar')
    
    # Объединение каналов цвета
    image_array = np.stack((red_channel, green_channel, blue_channel), axis=2)
    
    # Преобразование массива в изображение PIL
    image = Image.fromarray(image_array.astype(np.uint8))
    
    # Изменение размера изображения
    image = image.resize(image_size[::-1])
    
    return image

In [4]:
def frame_cut(video):
    """
    Функция режет видео на кадры и кладет их в папку
    video - путь до видеофайла
    """
    filename, _ = os.path.splitext(video)
    return_name = filename
    filename += '_cut'
    if not os.path.isdir(filename):
        os.mkdir(filename)
    
    video = VideoFileClip(video)
    # шаг, подходящий для фпс видео
    step = 1 / video.fps
    for cur_time in np.arange(0, video.duration, step):
        frame_filename = filename + "/frame" + str(cur_time) + ".jpg"
        video.save_frame(frame_filename, cur_time)

In [67]:
frame_cut('test.mp4')

In [5]:
def compression(path, quality_compr, quality_decompr):
    """
    Сжатие кадров видео
    """
    if not os.path.isdir("compressed_frames"):
        os.mkdir("compressed_frames")
    frame_count = 0    
    for filename in sorted(os.listdir(path), key=lambda x: os.path.getctime(os.path.join(path, x))):
    #sorted(glob.glob(path + '/*.jpg')):
        try:
            compressed_coeffs, image_size = compress_image(path + "/" + filename, quality_compr)
            # Восстановление сжатого изображения
            image_decompressed = decompress_image(compressed_coeffs, image_size , quality_decompr)
            image_decompressed.save("compressed_frames/"+str(frame_count) + ".jpg")
            frame_count += 1
        except:
            pass

In [7]:
compression("test_cut", 50, 30)

In [8]:
# Путь к папке с кадрами
frames_folder = 'compressed_frames/'

# Загрузка кадров из папки
frame_files = sorted(os.listdir(frames_folder), key=lambda x: os.path.getctime(os.path.join(frames_folder, x)))
#sorted(os.listdir(frames_folder))
frames = []
for file in frame_files:
    file_path = os.path.join(frames_folder, file)
    try:
        image = imread(file_path)
        frames.append(image)
    except:
        print(f"Не удалось прочитать файл {file_path}")

# Создание видео из кадров
video = ImageSequenceClip(frames, fps=60)

# Путь и имя файла видео
output_path = 'output.mp4'

# Сохранение видео
video.write_videofile(output_path, codec='libx264')


Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
